In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as data
import json
import numpy as np
import os
import itertools

In [20]:
class SelfAttention(nn.Module):

    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert (self.head_dim * heads == embed_size), "Embedding size needs to be divisible by heads"

        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)

    def forward(self, values, keys, query, mask):
        N = query.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        # Split the embedding into self.heads pieces
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        query = query.reshape(N, query_len, self.heads, self.head_dim)

        values = self.values(values)
        keys = self.keys(keys)
        queries = self.queries(query)

        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])

        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))

        attention = torch.softmax(energy / (self.embed_size ** (1/2)), dim=3)

        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(N, query_len, self.heads * self.head_dim)

        out = self.fc_out(out)
        return out

In [21]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size)
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        attention = self.attention(value, key, query, mask)

        x = self.dropout(self.norm1(attention + query))
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))
        return out

In [22]:
class Encoder(nn.Module):
    def __init__(self, src_vocab_size, embed_size, num_layers, heads, device, forward_expansion, dropout, max_length):
        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self.device = device
        self.word_embedding = nn.Embedding(src_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [TransformerBlock(embed_size, heads, dropout, forward_expansion) for _ in range(num_layers)]
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        out = self.dropout(self.word_embedding(x) + self.position_embedding(positions))

        for layer in self.layers:
            out = layer(out, out, out, mask)

        return out

In [23]:
class DecoderBlock(nn.Module):
    def __init__(self, embed_size, heads, forward_expansion, dropout, device):
        super(DecoderBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm = nn.LayerNorm(embed_size)
        self.transformer_block = TransformerBlock(embed_size, heads, dropout, forward_expansion)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, value, key, src_mask, trg_mask):
        attention = self.attention(x, x, x, trg_mask)
        query = self.dropout(self.norm(attention + x))
        out = self.transformer_block(value, key, query, src_mask)
        return out

In [24]:
class Decoder(nn.Module):
    def __init__(self, trg_vocab_size, embed_size, num_layers, heads, forward_expansion, dropout, device, max_length):
        super(Decoder, self).__init__()
        self.device = device
        self.word_embedding = nn.Embedding(trg_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [DecoderBlock(embed_size, heads, forward_expansion, dropout, device) for _ in range(num_layers)]
        )

        self.fc_out = nn.Linear(embed_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_out, src_mask, trg_mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        x = self.dropout(self.word_embedding(x) + self.position_embedding(positions))

        for layer in self.layers:
            x = layer(x, enc_out, enc_out, src_mask, trg_mask)

        out = self.fc_out(x)
        return out

In [25]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, trg_vocab_size, src_pad_idx, trg_pad_idx, embed_size=512, num_layers=6, forward_expansion=4, heads=8, dropout=0, device="cuda", max_length=100):
        super(Transformer, self).__init__()
        self.encoder = Encoder(src_vocab_size, embed_size, num_layers, heads, device, forward_expansion, dropout, max_length)
        self.decoder = Decoder(trg_vocab_size, embed_size, num_layers, heads, forward_expansion, dropout, device, max_length)
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device

    def make_src_mask(self, src):
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        return src_mask.to(self.device)

    def make_trg_mask(self, trg):
        N, trg_len = trg.shape
        trg_mask = torch.tril(torch.ones(trg_len, trg_len)).expand(N, 1, trg_len, trg_len)
        return trg_mask.to(self.device)

    def forward(self, src, trg):
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        enc_src = self.encoder(src, src_mask)
        out = self.decoder(trg, enc_src, src_mask, trg_mask)
        return out

In [26]:
notes_train = json.load(open('train_notes.json'))
notes_test = json.load(open('test_notes.json'))

In [70]:
def prepare_sequences(notes, note_to_int = None, sequence_length = 32):
    if note_to_int is None:
        pitch_names = sorted(set(itertools.chain(*notes)))
        # create a dictionary to map pitches to integers add pad tokens
        note_to_int = dict((note, number) for number, note in enumerate(pitch_names))
        note_to_int['<PAD>'] = len(note_to_int)
        note_to_int['<EOS>'] = len(note_to_int)
    int_to_note = {i: note for note, i in note_to_int.items()}
    sequences = []
    for song in notes:
        for i in range(len(song) - sequence_length):
            sequence = song[i:i+sequence_length]
            sequence = [note_to_int[note] for note in sequence]
            sequences.append(sequence)
    return sequences, note_to_int, int_to_note

In [71]:
def create_transformer_input_output(sequences):
    X, y = [], []
    for sequence in sequences:
        X.append(sequence[:-1])
        y.append(sequence[1:])
    return X, y

def create_dataloader(X, y, batch_size = 32):
    dataset = data.TensorDataset(torch.tensor(X), torch.tensor(y))
    return data.DataLoader(dataset, batch_size = batch_size, shuffle = True)

In [72]:
sequences, note_to_int, int_to_note = prepare_sequences(notes_train)

In [73]:
X, y = create_transformer_input_output(sequences)

In [74]:
train_dataloader = create_dataloader(X, y)

In [75]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [76]:
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0
    for i, (src, trg) in enumerate(iterator):
        src = src.to(device)
        trg = trg.to(device)
        optimizer.zero_grad()
        output = model(src, trg)
        output_dim = output.shape[-1]
        output = output[:, 1:].reshape(-1, output_dim)
        trg = trg[:, 1:].reshape(-1)
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(iterator)

In [77]:
model = Transformer(len(note_to_int), len(note_to_int), note_to_int['<PAD>'], note_to_int['<PAD>']).to(device)

In [78]:
train_dataloader = create_dataloader(X, y)

optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index = note_to_int['<PAD>'])
clip = 1

N_EPOCHS = 10
for epoch in range(N_EPOCHS):
    train_loss = train(model, train_dataloader, optimizer, criterion, clip)
    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Loss: {train_loss:.3f}')
    print()

torch.save(model.state_dict(), 'transformer.pth')

KeyboardInterrupt: 

In [95]:
def generate_notes(model, note_to_int, int_to_note, sequence_length = 32, n_notes = 100):
    model.eval()
    notes = []
    note = np.random.choice(list(note_to_int.values()))
    notes.append(note)
    for i in range(n_notes):
        sequence = torch.tensor(notes).unsqueeze(0).to(device)
        with torch.no_grad():
            output = model(sequence, sequence)
        output = output.argmax(2)[:,-1].item()
        notes.append(output)
    return [int_to_note[note] for note in notes]

In [96]:
generated_notes = generate_notes(model, note_to_int, int_to_note)

In [97]:
print(generated_notes)

['0.3.6.9', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1', 'C#-1']
